In [2]:
# Let's read in our document-term matrix
import pandas as pd
import pickle

data = pd.read_pickle('pickle/data_dtm.pkl')
data

"data = pd.read_pickle('pickle/data_dtm.pkl')\ndata"

In [4]:
# Import the necessary modules for LDA with gensim
from gensim import matutils, models
import scipy.sparse

# One of the required inputs is a term-document matrix
tdm = data.transpose()
tdm.head()

'# One of the required inputs is a term-document matrix\ntdm = data.transpose()\ntdm.head()'

In [6]:
# We're going to put the term-document matrix into a new gensim format, from df --> sparse matrix --> gensim corpus
sparse_counts = scipy.sparse.csr_matrix(tdm)
corpus = matutils.Sparse2Corpus(sparse_counts)

In [11]:
cv = pickle.load(open("cv_stop.pkl", "rb"))
id2word = dict((v, k) for k, v in cv.vocabulary_.items())

In [12]:
# Now that we have the corpus (term-document matrix) and id2word (dictionary of location: term),
# we need to specify two other parameters as well - the number of topics and the number of passes
lda = models.LdaModel(corpus=corpus, id2word=id2word, num_topics=2, passes=10)
lda.print_topics()

[(0,
  '0.025*"said" + 0.010*"coronavirus" + 0.010*"million" + 0.008*"company" + 0.008*"new" + 0.007*"year" + 0.007*"pandemic" + 0.007*"billion" + 0.006*"economy" + 0.006*"economic"'),
 (1,
  '0.018*"said" + 0.012*"coronavirus" + 0.012*"people" + 0.007*"time" + 0.007*"health" + 0.007*"new" + 0.005*"day" + 0.005*"newspaper" + 0.005*"just" + 0.005*"dont"')]

In [13]:
# LDA for num_topics = 3
lda = models.LdaModel(corpus=corpus, id2word=id2word, num_topics=3, passes=10)
lda.print_topics()


[(0,
  '0.040*"said" + 0.018*"coronavirus" + 0.010*"new" + 0.008*"government" + 0.007*"pandemic" + 0.007*"trump" + 0.007*"economy" + 0.006*"week" + 0.006*"china" + 0.006*"million"'),
 (1,
  '0.013*"said" + 0.013*"people" + 0.011*"coronavirus" + 0.008*"time" + 0.007*"newspaper" + 0.006*"dont" + 0.006*"health" + 0.006*"just" + 0.005*"day" + 0.005*"home"'),
 (2,
  '0.016*"company" + 0.009*"million" + 0.008*"business" + 0.007*"new" + 0.007*"financial" + 0.006*"data" + 0.006*"year" + 0.005*"time" + 0.005*"information" + 0.004*"revenue"')]

In [14]:
# LDA for num_topics = 4
lda = models.LdaModel(corpus=corpus, id2word=id2word, num_topics=4, passes=10)
lda.print_topics()

[(0,
  '0.013*"company" + 0.012*"million" + 0.011*"market" + 0.010*"year" + 0.009*"billion" + 0.009*"oil" + 0.009*"said" + 0.008*"quarter" + 0.008*"business" + 0.008*"financial"'),
 (1,
  '0.044*"said" + 0.018*"coronavirus" + 0.012*"government" + 0.010*"new" + 0.008*"people" + 0.008*"trump" + 0.007*"pandemic" + 0.007*"state" + 0.007*"crisis" + 0.007*"week"'),
 (2,
  '0.010*"time" + 0.009*"newspaper" + 0.008*"just" + 0.007*"dont" + 0.007*"said" + 0.007*"like" + 0.006*"coronavirus" + 0.005*"use" + 0.005*"people" + 0.005*"going"'),
 (3,
  '0.019*"people" + 0.016*"coronavirus" + 0.016*"health" + 0.015*"said" + 0.011*"virus" + 0.008*"care" + 0.007*"testing" + 0.007*"new" + 0.006*"day" + 0.006*"hospital"')]

In [27]:
# Create a function to pull out nouns from a string of text
from nltk import word_tokenize, pos_tag
import nltk

nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

def nouns(text):
    '''Given a string of text, tokenize the text and pull out only the nouns.'''
    is_noun = lambda pos: pos[:2] == 'NN'
    tokenized = word_tokenize(text)
    all_nouns = [word for (word, pos) in pos_tag(tokenized) if is_noun(pos)] 
    return ' '.join(all_nouns)

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/tobiaskarentiuskromanndahl/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/tobiaskarentiuskromanndahl/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


In [28]:
# Read in the cleaned data, before the CountVectorizer step
data_clean = pd.read_pickle('pickle/data_clean_r4.pkl')
data_clean

,content
0,so sorry to tell you that you have stage ovari...
1,if you buy something through link on this page...
2,cluster of more than pneumonia in the central ...
3,at hong international airport are being for my...
4,the finding that the outbreak of viral pneumon...
...,...
55532,good morning said the government had only ever...
55533,forward his future at despite talk that is con...
55534,the deal agreed by new jersey new york and isl...
55535,around the world have seen major and amid the ...


In [29]:
# Apply the nouns function to the transcripts to filter only on nouns
data_nouns = pd.DataFrame(data_clean.content.apply(nouns))
data_nouns.to_pickle('pickle/data_nouns.pkl')

,content
0,cancer i cancer treatment today life i disbeli...
1,something link page commission work people gin...
2,cluster pneumonia city member family world hea...
3,airport mystery illness china credit update st...
4,finding outbreak pneumonia china people family...
...,...
55532,morning government blanket order place home co...
55533,future talk swoop star mane side place bench w...
55534,deal york adjacent share data coronavirus mone...
55535,world mass life week mounting damage spread co...


In [6]:
# Create a new document-term matrix using only nouns
from sklearn.feature_extraction import text
from sklearn.feature_extraction.text import CountVectorizer

data_nouns = pd.read_pickle('pickle/data_nouns.pkl')

# Recreate a document-term matrix with only nouns
cvn = CountVectorizer(stop_words='english')
data_cvn = cvn.fit_transform(data_nouns.content)
data_dtmn = pd.DataFrame(data_cvn.toarray(), columns=cvn.get_feature_names())
data_dtmn.index = data_nouns.index
data_dtmn.to_pickle('pickle/data_dtmn.pkl')

In [7]:
data_dtmn = pd.read_pickle('pickle/data_dtmn.pkl')

# Create the gensim corpus
corpusn = matutils.Sparse2Corpus(scipy.sparse.csr_matrix(data_dtmn.transpose()))

# Create the vocabulary dictionary
id2wordn = dict((v, k) for k, v in cvn.vocabulary_.items())

In [33]:
# Let's start with 2 topics
ldan = models.LdaModel(corpus=corpusn, num_topics=2, id2word=id2wordn, passes=10)
ldan.print_topics()

[(0,
  '0.024*"people" + 0.023*"coronavirus" + 0.014*"time" + 0.014*"health" + 0.011*"day" + 0.011*"newspaper" + 0.009*"government" + 0.009*"virus" + 0.009*"home" + 0.007*"order"'),
 (1,
  '0.018*"coronavirus" + 0.015*"company" + 0.013*"year" + 0.012*"business" + 0.012*"economy" + 0.011*"market" + 0.009*"crisis" + 0.009*"week" + 0.008*"oil" + 0.008*"government"')]

In [34]:
# Let's try topics = 3
ldan = models.LdaModel(corpus=corpusn, num_topics=3, id2word=id2wordn, passes=10)
ldan.print_topics()

[(0,
  '0.023*"coronavirus" + 0.015*"economy" + 0.014*"year" + 0.012*"market" + 0.012*"government" + 0.011*"crisis" + 0.011*"oil" + 0.011*"week" + 0.010*"business" + 0.010*"quarter"'),
 (1,
  '0.028*"people" + 0.025*"coronavirus" + 0.017*"health" + 0.014*"time" + 0.013*"day" + 0.011*"virus" + 0.011*"government" + 0.010*"home" + 0.010*"newspaper" + 0.007*"world"'),
 (2,
  '0.016*"company" + 0.015*"season" + 0.015*"time" + 0.011*"league" + 0.011*"newspaper" + 0.010*"year" + 0.008*"summer" + 0.007*"order" + 0.007*"business" + 0.007*"team"')]

In [8]:
# Let's try 4 topics
ldan = models.LdaModel(corpus=corpusn, num_topics=4, id2word=id2wordn, passes=10)
ldan.print_topics()

[(0,
  '0.020*"time" + 0.018*"newspaper" + 0.012*"season" + 0.010*"order" + 0.010*"coronavirus" + 0.009*"express" + 0.009*"archive" + 0.009*"league" + 0.008*"way" + 0.008*"world"'),
 (1,
  '0.033*"people" + 0.030*"coronavirus" + 0.022*"health" + 0.015*"government" + 0.015*"virus" + 0.013*"day" + 0.011*"time" + 0.009*"home" + 0.009*"country" + 0.009*"number"'),
 (2,
  '0.016*"company" + 0.016*"business" + 0.015*"economy" + 0.015*"year" + 0.015*"coronavirus" + 0.014*"market" + 0.012*"quarter" + 0.011*"bank" + 0.011*"crisis" + 0.010*"stock"'),
 (3,
  '0.022*"oil" + 0.022*"coronavirus" + 0.016*"demand" + 0.014*"production" + 0.013*"company" + 0.013*"food" + 0.012*"china" + 0.012*"day" + 0.011*"travel" + 0.011*"week"')]

In [9]:
# Let's take a look at which topics each transcript contains
corpus_transformed = ldana[corpusna]
list(zip([a for [(a,b)] in corpus_transformed], data_dtmna.index))

NameError: name 'ldana' is not defined